In [2]:
# %%
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import keras_tuner as kt
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping

# --- Função para CARREGAR dados MICRO de um ticker ---
def carregar_dados_ticker_micro(ticker, pasta_dados_tratados):
    """
    Carrega os dados microeconômicos pré-processados de um arquivo CSV específico.

    Args:
        ticker (str): O nome do ticker (ex: "PETR4.SA").
        pasta_dados_tratados (str): O caminho para a pasta contendo os CSVs.

    Returns:
        tuple: (pd.DataFrame contendo os dados carregados,
                list de nomes das colunas de features,
                str com o nome da coluna target)
               Retorna (None, None, None) se o arquivo não for encontrado ou houver erro.
    """
    nome_arquivo = f"{ticker}_dados_unificados.csv"
    caminho_arquivo = os.path.join(pasta_dados_tratados, nome_arquivo)

    if not os.path.exists(caminho_arquivo):
        print(f"Erro: Arquivo não encontrado para {ticker} em {caminho_arquivo}")
        return None, None, None

    try:
        print(f"Carregando dados micro tratados de: {caminho_arquivo}")
        # A primeira coluna sem nome contém as datas, então tornamos ela o índice
        df = pd.read_csv(caminho_arquivo, index_col=0, parse_dates=True)

        # Verificar se as colunas necessárias existem
        colunas_esperadas = ['Preço', 'ROA', 'ROE', 'Margem Líquida', 'P/L', 'VP', 'Preço_anterior']
        colunas_faltantes = [col for col in colunas_esperadas if col not in df.columns]
        if colunas_faltantes:
            print(f"Aviso: Colunas faltantes no DataFrame para {ticker}: {colunas_faltantes}")
            return None, None, None

        # Criar colunas de feature e target
        # Similar ao código macro, precisamos definir o target como o preço atual
        # e as features como todas as outras colunas
        
        # Renomear 'Preço' para 'Close_Target' (mantendo consistência com o código macro)
        df['Close_Target'] = df['Preço']
        
        # Usar 'Preço_anterior' como 'Close_Feature' (preço em t-1)
        df['Close_Feature'] = df['Preço_anterior']
        
        # Definir target e features
        target_col = 'Close_Target'
        
        # Features são todas as colunas exceto o target ('Preço'/'Close_Target')
        feature_cols = [col for col in df.columns if col != 'Preço' and col != 'Close_Target']
        
        print(f"Colunas de Features identificadas para {ticker}: {feature_cols}")
        print(f"Coluna Target identificada para {ticker}: {target_col}")

        # Remover linhas onde o target é NaN
        df.dropna(subset=[target_col], inplace=True)

        return df, feature_cols, target_col

    except Exception as e:
        print(f"Erro ao carregar ou processar o arquivo {caminho_arquivo}: {e}")
        import traceback
        traceback.print_exc()
        return None, None, None

# --- Função para criar janelas MULTIVARIADAS (sem alterações) ---
def criar_janelas_multivariadas(features_array, target_array, janela):
    X, y = [], []
    if len(features_array) <= janela:
        print(f"Aviso em criar_janelas: comprimento dos dados ({len(features_array)}) não é maior que a janela ({janela}). Retornando vazio.")
        return np.array(X), np.array(y)
    for i in range(len(features_array) - janela):
        window_features = features_array[i:(i + janela), :]
        X.append(window_features)
        # Target correspondente: valor na linha i+janela no target_array
        # (que já representa T+1 em relação às features originais)
        y.append(target_array[i + janela])
    return np.array(X), np.array(y)


# --- Função build_model (sem alterações) ---
def build_model(hp, input_shape):
    model = keras.Sequential()
    for i in range(hp.Int('num_lstm_layers', 1, 2)):
        return_sequences = i < hp.Int('num_lstm_layers', 1, 2) - 1
        model.add(layers.LSTM(
            units=hp.Int(f'units_lstm_{i}', min_value=50, max_value=150, step=50),
            return_sequences=return_sequences,
            input_shape=input_shape if i == 0 else None
        ))
        model.add(layers.Dropout(hp.Float(f'dropout_lstm_{i}', 0.1, 0.3, step=0.1)))
    for i in range(hp.Int('num_dense_layers', 0, 1)):
        if hp.Int('num_dense_layers', 0, 1) > 0:
            model.add(layers.Dense(
                units=hp.Int(f'units_dense_{i}', min_value=32, max_value=64, step=32),
                activation='relu'
            ))
            model.add(layers.Dropout(hp.Float(f'dropout_dense_{i}', 0.1, 0.3, step=0.1)))
    model.add(layers.Dense(1))
    model.compile(
        optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-3, 5e-4])),
        loss='mean_squared_error'
    )
    return model

# --- Configurações ---

tickers = ["GGBR3.SA"]
pasta_dados_tratados = r"C:\Users\leona\pyhtonscripts\CodigoExperimentos\ExperimentoFeatures\dataframesMicro" 

start_date_val = "2020-01-01" 
end_date_val = "2022-12-31"  
start_date_test = "2023-01-01"
end_date_test = "2023-12-31"   

janela_min = 1
janela_max = 4
n_splits_cv = 5
max_trials_tuner = 10
epochs_tuner = 50
epochs_final = 100
resultados_dir = "resultados_lstm_cv_micro_tratados" # Novo diretório de resultados
os.makedirs(resultados_dir, exist_ok=True)

# --- Função de Experimento Específico para MICRO ---
def rodar_experimento_especifico_micro(ticker, janela, df_full, feature_cols, target_col):
    """
    Roda um experimento completo para um ticker e janela específicos,
    usando dados microeconômicos pré-carregados.
    """
    print(f"\n--- Iniciando Experimento com Dados Micro Tratados: {ticker}, Janela {janela} ---")

    base_filename = f"{resultados_dir}/{ticker}_Janela_{janela}"
    metrics_path = f"{base_filename}_metrics.csv"
    hiperparametros_path = f"{base_filename}_hiperparametros.csv"
    grafico_path = f"{base_filename}_grafico_teste_final.png"
    previsoes_path = f"{base_filename}_previsoes_teste_final.csv"

    if os.path.exists(metrics_path):
        print(f"Resultados já existem para {ticker}, Janela {janela}. Pulando...")
        return

    # 1. Separar Dados de Treino/Validação (2020-2022) e Teste (2023)
    try:
        df_val_train = df_full.loc[start_date_val:end_date_val].copy()
        df_test_final = df_full.loc[start_date_test:end_date_test].copy()
    except KeyError as e:
        print(f"Erro ao dividir dados por data para {ticker}: {e}. Verifique as datas e o índice do DataFrame.")
        return

    if df_val_train.empty or df_test_final.empty:
        print(f"Erro: Período de treino/validação ou teste está vazio para {ticker}. Verifique as datas e os dados.")
        return

    # Verificar se há NaNs nas features ou target após o split
    if df_val_train[feature_cols].isnull().values.any() or df_val_train[target_col].isnull().values.any():
        print(f"Aviso: Encontrados NaNs nas features ou target de treino/validação para {ticker}. Verifique o pré-processamento.")
        return

    if df_test_final[feature_cols].isnull().values.any() or df_test_final[target_col].isnull().values.any():
        print(f"Aviso: Encontrados NaNs nas features ou target de teste para {ticker}. Verifique o pré-processamento.")
        return


    # Separar features e target para treino/validação
    features_val_train = df_val_train[feature_cols]
    target_val_train = df_val_train[[target_col]] # Manter como DataFrame para o scaler

    # 2. Escalonamento (DOIS SCALERS)
    scaler_features = MinMaxScaler()
    scaler_target = MinMaxScaler() # Scaler dedicado para o target ('Close_Target')

    # Ajustar (fit) os scalers SOMENTE nos dados de treino/validação (2020-2022)
    scaled_features_val_train = scaler_features.fit_transform(features_val_train)
    # Ajustar e transformar o target T+1 usando o scaler dedicado
    scaled_target_val_train = scaler_target.fit_transform(target_val_train)

    num_features = scaled_features_val_train.shape[1]
    print(f"Número de features para o modelo: {num_features}")

    # 3. Criar Janelas Multivariadas para Treino/Validação
    X_val_train_full, y_val_train_full = criar_janelas_multivariadas(
        scaled_features_val_train,
        scaled_target_val_train.flatten(), # Passar target escalado como array 1D
        janela
    )

    if X_val_train_full.size == 0 or y_val_train_full.size == 0:
        print(f"Erro: Não foi possível criar janelas de treino/validação para {ticker}, Janela {janela}. Verifique o tamanho dos dados e a janela.")
        return

    # 4. Otimização de Hiperparâmetros (Keras Tuner)
    print("Iniciando busca de hiperparâmetros...")
    input_shape = (janela, num_features) # Forma da entrada para LSTM
    tuner = kt.RandomSearch(
        lambda hp: build_model(hp, input_shape=input_shape),
        objective='val_loss',
        max_trials=max_trials_tuner,
        executions_per_trial=1,
        directory='keras_tuner_cv_micro_tratados', # Novo diretório
        project_name=f'LSTM_{ticker}_Janela_{janela}',
        overwrite=True
    )

    early_stopping_tuner = EarlyStopping(monitor='val_loss', patience=10, verbose=0, restore_best_weights=True)

    # Dividir para validação interna do tuner (respeitando ordem)
    split_index = int(len(X_val_train_full) * 0.8)
    X_tuner_train, y_tuner_train = X_val_train_full[:split_index], y_val_train_full[:split_index]
    X_tuner_val, y_tuner_val = X_val_train_full[split_index:], y_val_train_full[split_index:]

    if len(X_tuner_val) == 0:
        print(f"Aviso: Sem dados de validação suficientes para tuner em {ticker}, Janela {janela}. Usando treino completo para busca.")
        tuner.search(X_val_train_full, y_val_train_full, epochs=epochs_tuner, callbacks=[early_stopping_tuner], verbose=1)
    else:
        tuner.search(X_tuner_train, y_tuner_train, epochs=epochs_tuner,
                     validation_data=(X_tuner_val, y_tuner_val),
                     callbacks=[early_stopping_tuner], verbose=1)

    try:
        best_hps = tuner.get_best_hyperparameters(1)[0]
        print("\nMelhores Hiperparâmetros encontrados:")
        print(best_hps.values)
    except IndexError:
        print(f"Erro: Keras Tuner não encontrou nenhum hiperparâmetro válido para {ticker}, Janela {janela}. Pulando para próximo.")
        return # Pula o resto do experimento se não houver HPs

    # 5. Avaliação Cruzada dos Melhores HPs (TimeSeriesSplit no período 2020-2022)
    print(f"\nIniciando TimeSeriesSplit CV ({n_splits_cv} folds) para avaliar os melhores HPs...")
    tscv = TimeSeriesSplit(n_splits=n_splits_cv)
    cv_metrics = {'mae': [], 'mse': [], 'rmse': [], 'r2': []}
    fold = 0
    for train_index, val_index in tscv.split(X_val_train_full): # Split nas janelas criadas
        fold += 1
        print(f"   Fold {fold}/{n_splits_cv}...")
        X_train_fold, X_val_fold = X_val_train_full[train_index], X_val_train_full[val_index]
        y_train_fold, y_val_fold = y_val_train_full[train_index], y_val_train_full[val_index]

        # Verificar se há dados suficientes no fold
        if len(X_train_fold) == 0 or len(X_val_fold) == 0:
            print(f"   Aviso: Fold {fold} tem 0 amostras de treino ou validação. Pulando fold.")
            continue

        model_cv = tuner.hypermodel.build(best_hps)
        early_stopping_cv = EarlyStopping(monitor='loss', patience=10, verbose=0)
        model_cv.fit(X_train_fold, y_train_fold, epochs=epochs_final, batch_size=32,
                     callbacks=[early_stopping_cv], verbose=0)

        # Avaliar no conjunto de validação do fold
        previsoes_val_fold_scaled = model_cv.predict(X_val_fold)
        # INVERTER usando SCALER_TARGET
        previsoes_val_fold = scaler_target.inverse_transform(previsoes_val_fold_scaled)
        reais_val_fold_scaled = y_val_fold.reshape(-1, 1)
        reais_val_fold = scaler_target.inverse_transform(reais_val_fold_scaled)

        # Calcular métricas do fold
        cv_metrics['mae'].append(mean_absolute_error(reais_val_fold, previsoes_val_fold))
        cv_metrics['mse'].append(mean_squared_error(reais_val_fold, previsoes_val_fold))
        cv_metrics['rmse'].append(np.sqrt(cv_metrics['mse'][-1]))
        try:
            r2_fold = r2_score(reais_val_fold, previsoes_val_fold)
            cv_metrics['r2'].append(r2_fold if np.isfinite(r2_fold) else -1.0)
        except ValueError:
            cv_metrics['r2'].append(-1.0)

        print(f"     Fold {fold} MAE: {cv_metrics['mae'][-1]:.4f}, RMSE: {cv_metrics['rmse'][-1]:.4f}, R2: {cv_metrics['r2'][-1]:.4f}")

    # Calcular métricas médias da validação cruzada (apenas dos folds válidos)
    if cv_metrics['mae']: # Verificar se algum fold foi executado
        avg_cv_mae = np.mean(cv_metrics['mae'])
        avg_cv_mse = np.mean(cv_metrics['mse'])
        avg_cv_rmse = np.mean(cv_metrics['rmse'])
        avg_cv_r2 = np.mean(cv_metrics['r2'])
        print("\nResultados Médios da Validação Cruzada (TimeSeriesSplit):")
        print(f"   Avg MAE: {avg_cv_mae:.4f}, Avg MSE: {avg_cv_mse:.4f}, Avg RMSE: {avg_cv_rmse:.4f}, Avg R2: {avg_cv_r2:.4f}")
    else:
        print("\nAviso: Nenhum fold da validação cruzada foi completado com sucesso.")
        avg_cv_mae, avg_cv_mse, avg_cv_rmse, avg_cv_r2 = np.nan, np.nan, np.nan, np.nan


    # 6. Treinamento Final (com melhores HPs, usando TODAS as janelas de 2020-2022)
    print("\nTreinando modelo final com melhores HPs...")
    model_final = tuner.hypermodel.build(best_hps)
    early_stopping_final = EarlyStopping(monitor='loss', patience=15, verbose=1, restore_best_weights=True)
    history = model_final.fit(X_val_train_full, y_val_train_full, epochs=epochs_final, batch_size=32,
                              callbacks=[early_stopping_final], verbose=1)

    # Verificar se o treinamento convergiu (loss diminuiu)
    if not history.history or not history.history['loss'] or min(history.history['loss']) == history.history['loss'][0]:
         print(f"Aviso: Treinamento final para {ticker}, Janela {janela} pode não ter convergido bem (loss: {history.history['loss'][-1] if history.history['loss'] else 'N/A'}).")


    # 7. Avaliação Final no Conjunto de Teste (2023 - Hold-Out)
    print("\nAvaliando modelo final no conjunto de teste (2023)...")

    # Separar features e target para teste
    features_test_final = df_test_final[feature_cols]
    target_test_final = df_test_final[[target_col]] # DataFrame

    # Escalar features de teste USANDO scaler_features AJUSTADO NO TREINO
    scaled_features_test_final = scaler_features.transform(features_test_final)
    # Escalar target de teste USANDO scaler_target AJUSTADO NO TREINO
    scaled_target_test_final = scaler_target.transform(target_test_final)

    # Criar janelas de teste
    X_test_final, y_test_final = criar_janelas_multivariadas(
        scaled_features_test_final,
        scaled_target_test_final.flatten(),
        janela
    )

    if X_test_final.size == 0 or y_test_final.size == 0:
        print(f"Erro: Não foi possível criar janelas de teste final para {ticker}, Janela {janela}. Verifique datas/janela/tamanho dos dados de teste.")
        # Salvar métricas parciais se a CV rodou
        if not np.isnan(avg_cv_mae):
            metrics_df = pd.DataFrame([{
                'Ticker': ticker, 'Janela': janela, 'Num Features': num_features,
                'CV Avg MAE': avg_cv_mae, 'CV Avg MSE': avg_cv_mse, 'CV Avg RMSE': avg_cv_rmse, 'CV Avg R2': avg_cv_r2,
                'Teste MAE': np.nan, 'Teste MSE': np.nan, 'Teste RMSE': np.nan, 'Teste R2': np.nan
             }])
            metrics_df.to_csv(metrics_path, index=False)
            print(f"Métricas parciais (CV) salvas em {metrics_path}")
        return # Não continuar se não houver janelas de teste

    # Fazer previsões (escaladas)
    previsoes_test_scaled = model_final.predict(X_test_final)

    # INVERTER usando SCALER_TARGET
    previsoes_test_final = scaler_target.inverse_transform(previsoes_test_scaled)
    reais_test_final_scaled = y_test_final.reshape(-1, 1)
    reais_test_final = scaler_target.inverse_transform(reais_test_final_scaled)

    # Calcular métricas do teste final
    mae_test = mean_absolute_error(reais_test_final, previsoes_test_final)
    mse_test = mean_squared_error(reais_test_final, previsoes_test_final)
    rmse_test = np.sqrt(mse_test)
    r2_test = r2_score(reais_test_final, previsoes_test_final)
    print("\nResultados da Avaliação Final (Teste 2023):")
    print(f"   MAE Teste: {mae_test:.4f}, MSE Teste: {mse_test:.4f}, RMSE Teste: {rmse_test:.4f}, R² Teste: {r2_test:.4f}")

    # 8. Salvar Resultados
    print("\nSalvando resultados...")
    # Métricas
    metrics_df = pd.DataFrame([{
        'Ticker': ticker, 'Janela': janela, 'Num Features': num_features,
        'CV Avg MAE': avg_cv_mae, 'CV Avg MSE': avg_cv_mse, 'CV Avg RMSE': avg_cv_rmse, 'CV Avg R2': avg_cv_r2,
        'Teste MAE': mae_test, 'Teste MSE': mse_test, 'Teste RMSE': rmse_test, 'Teste R2': r2_test
    }])
    metrics_df.to_csv(metrics_path, index=False)
    print(f"Métricas salvas em {metrics_path}")

    # Hiperparâmetros
    hiperparametros_df = pd.DataFrame([best_hps.values])
    hiperparametros_df.to_csv(hiperparametros_path, index=False)
    print(f"Hiperparâmetros salvos em {hiperparametros_path}")

    # Previsões do teste final
    # As datas corretas para y_test_final são as datas do df_test_final a partir da posição 'janela'
    # Pois a primeira janela usa dados de 0 a janela-1 para prever o target em 'janela'
    datas_teste_final = df_test_final.index[janela:]

    # Verificar se o número de datas corresponde ao número de previsões/reais
    if len(datas_teste_final) == len(reais_test_final) and len(datas_teste_final) == len(previsoes_test_final):
        previsoes_df = pd.DataFrame({
            'Data': datas_teste_final,
            'Preço Real': reais_test_final.flatten(),
            'Preço Previsto': previsoes_test_final.flatten()
        })
        previsoes_df.to_csv(previsoes_path, index=False)
        print(f"Previsões do teste salvas em {previsoes_path}")

        # Gerar gráfico
        plt.figure(figsize=(14, 7))
        plt.plot(previsoes_df['Data'], previsoes_df['Preço Real'], label='Real (Teste 2023)', color='blue', linewidth=1.5)
        plt.plot(previsoes_df['Data'], previsoes_df['Preço Previsto'], label='Previsto (Teste 2023)', color='orange', linestyle='--', linewidth=1.5)
        plt.title(f'Preços Reais vs Previstos (Teste Final com Dados Micro) - {ticker} (Janela {janela})')
        plt.xlabel('Data')
        plt.ylabel('Preço')
        plt.legend()
        plt.grid(alpha=0.4)
        plt.tight_layout()
        plt.savefig(grafico_path)
        print(f"Gráfico do teste salvo em {grafico_path}")
        plt.close() # Fechar a figura para liberar memória
    else:
        print(f"Erro: Discrepância entre número de datas ({len(datas_teste_final)}) e previsões/reais ({len(reais_test_final)}/{len(previsoes_test_final)}) para {ticker}, Janela {janela} no teste final.")
        print(f"   Índice df_test_final começa: {df_test_final.index.min()}, termina: {df_test_final.index.max()}, len: {len(df_test_final)}")
        print(f"   Índice datas_teste_final começa: {datas_teste_final.min() if not datas_teste_final.empty else 'N/A'}, termina: {datas_teste_final.max() if not datas_teste_final.empty else 'N/A'}, len: {len(datas_teste_final)}")
        print(f"   Tamanho X_test_final: {X_test_final.shape}, y_test_final: {y_test_final.shape}, previsoes_test_final: {previsoes_test_final.shape}")
        print("   Gráfico/Previsões não salvos devido à inconsistência.")

    print(f"--- Experimento Concluído: {ticker}, Janela {janela} ---")


# --- Loop Principal de Experimentos para MICRO ---
def rodar_experimentos_cv_micro_tratados():
    for ticker in tickers:
        # Carregar os dados para o ticker atual
        df_ticker_full, feature_cols, target_col = carregar_dados_ticker_micro(ticker, pasta_dados_tratados)

        if df_ticker_full is None:
            print(f"Não foi possível carregar dados para {ticker}. Pulando ticker.")
            continue # Pula para o próximo ticker se os dados não puderam ser carregados

        # Iterar pelas janelas para o ticker atual
        for janela in range(janela_min, janela_max + 1):
            try:
                # Passar o DataFrame completo e as colunas identificadas
                rodar_experimento_especifico_micro(ticker, janela, df_ticker_full, feature_cols, target_col)
            except Exception as e:
                print(f"Erro CRÍTICO ao rodar experimento {ticker}, Janela {janela}: {e}")
                import traceback
                traceback.print_exc() # Imprime o stack trace completo para depuração

# --- Rodar os experimentos ---
if __name__ == "__main__":
    # Verificar se a pasta de dados existe
    if not os.path.isdir(pasta_dados_tratados):
         print(f"ERRO FATAL: A pasta de dados micro tratados '{pasta_dados_tratados}' não foi encontrada.")
         print("Por favor, verifique o caminho na variável 'pasta_dados_tratados'.")
    else:
        rodar_experimentos_cv_micro_tratados()
        print("\n--- TODOS OS EXPERIMENTOS COM DADOS MICRO TRATADOS CONCLUÍDOS ---")

Trial 10 Complete [00h 00m 11s]
val_loss: 0.0015139520401135087

Best val_loss So Far: 0.0012230363208800554
Total elapsed time: 00h 01m 32s

Melhores Hiperparâmetros encontrados:
{'num_lstm_layers': 1, 'units_lstm_0': 100, 'dropout_lstm_0': 0.1, 'num_dense_layers': 1, 'learning_rate': 0.001, 'units_lstm_1': 100, 'dropout_lstm_1': 0.2, 'units_dense_0': 64, 'dropout_dense_0': 0.1}

Iniciando TimeSeriesSplit CV (5 folds) para avaliar os melhores HPs...
   Fold 1/5...
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
     Fold 1 MAE: 0.3093, RMSE: 0.5263, R2: 0.8584
   Fold 2/5...


c:\Users\leona\anaconda3\envs\ambiente\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
     Fold 2 MAE: 1.6835, RMSE: 1.8177, R2: 0.2603
   Fold 3/5...


c:\Users\leona\anaconda3\envs\ambiente\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
     Fold 3 MAE: 0.4456, RMSE: 0.5670, R2: 0.6917
   Fold 4/5...


...

6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
     Fold 4 MAE: 1.9427, RMSE: 2.0324, R2: -1.4906
   Fold 5/5...


c:\Users\leona\anaconda3\envs\ambiente\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
     Fold 5 MAE: 0.4171, RMSE: 0.5610, R2: 0.9468

Resultados Médios da Validação Cruzada (TimeSeriesSplit):
   Avg MAE: 0.9596, Avg MSE: 1.6696, Avg RMSE: 1.1009, Avg R2: 0.2533

Treinando modelo final com melhores HPs...
Epoch 1/100


c:\Users\leona\anaconda3\envs\ambiente\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - loss: 0.1030
Epoch 2/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0106
Epoch 3/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0079
Epoch 4/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0058
Epoch 5/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0057
Epoch 6/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 0.0053
Epoch 7/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0045
Epoch 8/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0049
Epoch 9/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0048
Epoch 10/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0041
Epoch 11/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0040
Epoch 12/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0043
Epoch 13/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0040
Epoch 14/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.0039
Epoch 15/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.0042
Epoch 16/100
35/